### BATINA AGASA
#### Homework 3

In [1]:
import requests
import re
from collections import defaultdict
from lxml import etree
import tqdm
import time

def xml_to_dict(tree, paths=None, nsmap=None, strip_ns=False):
    """Convert an XML tree to a dictionary.
    :param tree: etree Element
    :type tree: :class:`lxml.etree._Element`
    :param paths: An optional list of XPath expressions applied on the XML tree.
    :type paths: list[basestring]
    :param nsmap: An optional prefix-namespace mapping for conciser spec of paths.
    :type nsmap: dict
    :param strip_ns: Flag for whether to remove the namespaces from the tags.
    :type strip_ns: bool
    """
    paths = paths or ['.//']
    nsmap = nsmap or {}
    fields = defaultdict(list)
    for path in paths:
        elements = tree.findall(path, nsmap)
        for element in elements:
            tag = re.sub(
                r'\{.*\}', '', element.tag) if strip_ns else element.tag
            fields[tag].append(element.text)
    return dict(fields)

sets = ["physics","econ","cs"]
XMLParser = etree.XMLParser(remove_blank_text=True, recover=True, resolve_entities=False)


for set_ in tqdm.tqdm(sets):
    response = requests.get("http://export.arxiv.org/oai2?verb=ListIdentifiers&set={}&from=2020-01-01&metadataPrefix=oai_dc".format(set_))
    tree = etree.XML(response.content, parser=XMLParser)
    papers = xml_to_dict(tree=tree)
    ids = [id_.split(":")[-1] for id_ in papers["{http://www.openarchives.org/OAI/2.0/}identifier"]]
    arxiv_txt = open(r'C:\Users\ferve\Documents\NoSQL\Homeworks\arxiv_db.txt', 'a')
    [arxiv_txt.write(id_ + "\n") for id_ in ids]
    arxiv_txt.close()
    if len(papers) == 7:
        time.sleep(25)
        continue
    token = papers["{http://www.openarchives.org/OAI/2.0/}resumptionToken"][0]
    time.sleep(25)
    done = False
    while done == False:
            response = requests.get("http://export.arxiv.org/oai2?verb=ListIdentifiers&resumptionToken={}".format(token))
            tree = etree.XML(response.content, parser=XMLParser)
            papers = xml_to_dict(tree=tree)
            ids = [id_.split(":")[-1] for id_ in papers["{http://www.openarchives.org/OAI/2.0/}identifier"]]
            if len(ids) != 10000:
                done = True
            arxiv_txt = open(r'C:\Users\ferve\Documents\NoSQL\Homeworks\arxiv_db.txt', 'a')
            [arxiv_txt.write(id_ + "\n") for id_ in ids]
            arxiv_txt.close()
            if len(papers) == 7:
                done = True
            else:
                token = papers["{http://www.openarchives.org/OAI/2.0/}resumptionToken"][0]
            time.sleep(25)

100%|██████████| 3/3 [17:49<00:00, 356.54s/it]


ModuleNotFoundError: No module named 'feedparser'

In [5]:
import requests
import feedparser
import tqdm
import time
import pymongo

client = pymongo.MongoClient('localhost', 27017)
mydb = client["Homeworks_2"]
collection = mydb["Homework2"]


with open(r'C:\Users\ferve\Documents\NoSQL\Homeworks\arxiv_db.txt',"r") as lines:
    ids = lines.read().split("\n")[0:-2]

results = {}
ids_query = []
iteration = 1

for id_ in tqdm.tqdm(ids):
    if iteration % 100 != 0:
        iteration += 1
        ids_query.append(id_)
    else:
        ids_query = ",".join(ids_query)
        response = requests.get('http://export.arxiv.org/api/query?id_list={}&max_results=100'.format(ids_query))
        feed = feedparser.parse(response.content)
        list_of_insertion = []
        for entry in feed.entries:
            list_of_insertion.append(dict(entry))
        collection.insert_many(list_of_insertion)
        ids_query = []
        iteration = 1
        time.sleep(1/3)
        

 53%|█████▎    | 217799/408542 [1:32:26<1:20:57, 39.26it/s]


KeyboardInterrupt: 

In [1]:
import pymongo
import requests
import feedparser
import tqdm
import time


client = pymongo.MongoClient('localhost', 27017)
mydb = client["Homeworks_2"]
collection = mydb["Homework2"]

### Question 1

In [2]:
collection.create_index([ ("author",-1) ]) # on a fait un index sur la variable autheur, le -1 correcpond au plus vieux au plus jeune

'author_-1'

### Question 2

In [4]:

fervent= collection.count_documents(
    {
        'published' : 
        {'$not' : 
         {'$regex' : ".*2019.*|.*2020.*|.*2021.*|.*2018.*|.*2017.*|.*2016.*|.*2015.*|.*2022.*"}
        }
    }
)
print(fervent)

3151


In [5]:
#Nous supprimerons les élements (delete_many) qui ne contiennent pas les années 2015.


collection.delete_many(
    {
        'published' : 
        {'$not' : 
         {'$regex' : ".*2015.*|.*2016.*|.*2017.*|.*2018.*|.*2019.*|.*2020.*|.*2021.*|.*2022.*"}
        }
    }
)

#On a pu supprimer 3151 éléments

### Question 3

In [6]:
# Avec seul auteur

doc_avec_1auth = collection.count_documents({"authors" :{"$size":1}})
print("Il y'a " + str(doc_avec_1auth) + " documents comportant un seul auteur")

 # Avec deux auteurs
doc_avec_2auth = collection.count_documents({"authors" :{"$size":2}})
print("Il y'a " + str(doc_avec_2auth) +" documents comportant deux auteurs")

Il y'a 27168 documents comportant un seul auteur
Il y'a 42887 documents comportant deux auteurs


### Question 4

In [17]:
# On a juste pris le premier de la liste vu que MongoDB insère +1 aux id

dernier = collection.find().sort("_id", 1).limit(1)
print(list(dernier))

[{'_id': ObjectId('6261b5c41f507438547e6915'), 'id': 'http://arxiv.org/abs/1501.00434v2', 'guidislink': True, 'link': 'http://arxiv.org/abs/1501.00434v2', 'updated': '2016-01-12T19:33:34Z', 'updated_parsed': [2016, 1, 12, 19, 33, 34, 1, 12, 0], 'published': '2015-01-02T16:31:20Z', 'published_parsed': [2015, 1, 2, 16, 31, 20, 4, 2, 0], 'title': 'Monetary Policy and Dark Corners in a stylized Agent-Based Model', 'title_detail': {'type': 'text/plain', 'language': None, 'base': '', 'value': 'Monetary Policy and Dark Corners in a stylized Agent-Based Model'}, 'summary': 'We extend in a minimal way the stylized model introduced in in "Tipping\nPoints in Macroeconomic Agent Based Models" [JEDC 50, 29-61 (2015)], with the\naim of investigating the role and efficacy of monetary policy of a `Central\nBank\' that sets the interest rate such as to steer the economy towards a\nprescribed inflation and employment level. Our major finding is that provided\nits policy is not too aggressive (in a sense

### Question 5

In [9]:
q5 = collection.count_documents({'title': {'$regex' : "technology", "$options" : 'i' }})
print("Il y'a " + str(q5) + " documents parlant de technology donc on va pas print sinon ça va flood.")

#for docs_sans_mesh in meshword_null :
#    print(docs_sans_mesh)

# il y'a 22065 documents

Il y'a 195 documents parlant de technology donc on va pas print sinon ça va flood.


### Question 6

In [10]:
arxiv_comment_null = collection.count_documents({'arxiv_comment': None })
print("Il y'a " + str(arxiv_comment_null) + " documents sans arxiv_comment.")

Il y'a 52055 documents sans arxiv_comment.


### Question 7

In [11]:
arxiv_affi = collection.count_documents({'arxiv_affiliation' : {'$not': {'$regex' : ""}}})
print("Il y'a " + str(arxiv_affi ) + " documents avec arxiv_affiliation.")

arxiv_affi2 = collection.count_documents({'arxiv_affiliation' : {'$regex' : ""}})
print("Il y'a " + str(arxiv_affi2) + " documents sans arxiv_affiliation.")

Il y'a 204485 documents avec arxiv_affiliation.
Il y'a 7986 documents sans arxiv_affiliation.


### Question 8

In [12]:


question_8 = collection.count_documents(
    {"published":{"$regex" : ".*2021.*|.*2020.*|.*2022.*"}})

print("Il y'a " + str(question_8) + " documents sortis après 2019")

Il y'a 171278 documents sortis après 2019


### Question 9

In [13]:
question_9 = collection.count_documents({'arxiv_affiliation': {'$regex' : ".*China.*", '$options' :'i' }})
print("Il y'a " + str(question_9) + " documents avec China en arxiv_affiliation")

Il y'a 110 documents avec China en arxiv_affiliation


### Question 10

In [14]:
pray_the_stackoverflow_guy= collection.aggregate(
    [
        {"$group": {"_id": "$id", "unique_ids": {"$addToSet": "$_id"}, "count": {"$sum": 1}}},
        {"$match": {"count": { "$gte": 2 }}}
    ]
)

response = []
for doc in pray_the_stackoverflow_guy:
    del doc["unique_ids"][1]
    for id in doc["unique_ids"]:
        response.append(id)
        
collection.delete_many({"_id": {"$in": response}})

### Question 11

In [15]:
collection.delete_many(
    {"summary" : {"$regex" : '\"R.*'}})
#211822
#211746   

### Question 13

In [ ]:


collection.count_documents({'title': {'$regex': ".*environment.*", '$options': 'i' }})